### Extract the data of patients diagnosed with vascular dementia

In [2]:
UKBB_PATH = "~/biobank/ukb672220.csv"

from DataFields import DataFields

datafields = DataFields


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Print all columns to view available columns
columns = pd.read_csv(UKBB_PATH, nrows=0).columns.tolist()
print(columns)

['eid', '3-0.0', '3-1.0', '3-2.0', '3-3.0', '4-0.0', '4-1.0', '4-2.0', '4-3.0', '5-0.0', '5-1.0', '5-2.0', '5-3.0', '6-0.0', '6-1.0', '6-2.0', '6-3.0', '19-0.0', '21-0.0', '21-1.0', '21-2.0', '21-3.0', '31-0.0', '34-0.0', '36-0.0', '36-1.0', '36-2.0', '36-3.0', '37-0.0', '37-1.0', '37-2.0', '37-3.0', '38-0.0', '38-1.0', '38-2.0', '38-3.0', '39-0.0', '39-1.0', '39-2.0', '39-3.0', '40-0.0', '40-1.0', '40-2.0', '40-3.0', '41-0.0', '41-1.0', '41-2.0', '41-3.0', '42-0.0', '42-1.0', '42-2.0', '42-3.0', '43-0.0', '43-1.0', '43-2.0', '43-3.0', '44-0.0', '44-1.0', '44-2.0', '44-3.0', '45-0.0', '45-1.0', '45-2.0', '45-3.0', '46-0.0', '46-1.0', '46-2.0', '46-3.0', '47-0.0', '47-1.0', '47-2.0', '47-3.0', '48-0.0', '48-1.0', '48-2.0', '48-3.0', '49-0.0', '49-1.0', '49-2.0', '49-3.0', '50-0.0', '50-1.0', '50-2.0', '50-3.0', '51-0.0', '51-1.0', '51-2.0', '51-3.0', '52-0.0', '53-0.0', '53-1.0', '53-2.0', '53-3.0', '54-0.0', '54-1.0', '54-2.0', '54-3.0', '55-0.0', '55-1.0', '55-2.0', '55-3.0', '77-0.0'

In [5]:

# Read the chunk and get columns from the datafields dictionary
# if Oldest=True, take the oldest instance, otherwise newest
def get_columns_from_chunk(chunk, datafields, oldest=False):
    selected_columns = {}
    for field_name, instances in datafields.items():
        instance_key = min(instances) if oldest else max(instances)
        selected_columns[field_name] = instances[instance_key]
    
    # Select only the necessary columns from the chunk
    filtered_chunk = chunk[list(selected_columns.values())].rename(columns={
        v: k for k, v in selected_columns.items()
    })
    
    return filtered_chunk
    

In [6]:
def filter_by_vascular_dementia(chunk):
    col_1_filter = "Vascular Dementia Date First Reported"
    col_2_filter = "Date of vascular dementia report"
    col_combined = "Vascular Dementia Report Date"
    Y_col = "Has Vascular Dementia"
    
    # Filter rows where either col_1_filter or col_2_filter is not NaN
    filtered_chunk = chunk[chunk[col_1_filter].notna() | chunk[col_2_filter].notna()].copy()  # Make a copy here
    
    # Combine dates and add the new columns
    filtered_chunk.loc[:, col_combined] = pd.to_datetime(
        filtered_chunk[col_1_filter].combine_first(filtered_chunk[col_2_filter]), errors="coerce"
    )
    
    # Filter by dates to remove unspecified entries
    filtered_chunk.loc[:, Y_col] = (
        filtered_chunk[col_combined]
        .between(pd.Timestamp("1950-01-01"), pd.Timestamp("2030-12-31"))
        .astype(int)
    )

    return filtered_chunk



In [7]:

# from the given "fields" list, convert all columns where date is in range, to 0 or 1 instead of a date.
def convert_date_to_binary(chunk, fields):
    start_date = pd.Timestamp("1950-01-01")
    end_date = pd.Timestamp("2030-01-01")
    
    for col in fields:
        df[col] = pd.to_datetime(df[col], errors='coerce')

        mask = (df[col] >= start_date) & (df[col] <= end_date)

        df[col] = np.where(mask, 1, 0)
    
    return df

### Extract rows diagnosed with vascular dementia through the use of two fields

In [6]:

chunk_size = 10000

# Create an empty DataFrame to hold the filtered data
result_df = pd.DataFrame()

for chunk in pd.read_csv(UKBB_PATH, chunksize=chunk_size, low_memory=False):

    filtered_chunk = get_columns_from_chunk(chunk, datafields, oldest=True)

    filtered_chunk = filter_by_vascular_dementia(filtered_chunk)
    
    # Append to the result DataFrame
    result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)

# Save the result
result_df.to_csv('vascular_dementia_filtered.csv')

# Display the first 30 rows
result_df.head(5)

,id,Birth Year,Sex,Ethnicity,Education,Primary Hypertension,Secondary Hypertension,Vascular Dementia Date First Reported,Date of vascular dementia report,BMI Impedance,...,Red blood cell (erythrocyte) count,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Vascular Dementia Report Date,Has Vascular Dementia
0,1002012,1940.0,1,1001.0,1.0,2009-08-13,NaN,2019-09-06,2019-09-06,27.0801,...,4.799,13.07,0.050,1.047,7.79,75.0,182.0,62.0,2019-09-06,1
1,1002324,1938.0,1,1001.0,-7.0,2000-10-01,NaN,2022-07-01,NaN,33.7984,...,4.080,15.80,0.134,3.280,3.60,84.0,146.0,73.0,2022-07-01,1
2,1005163,1941.0,0,1001.0,1.0,NaN,NaN,2022-09-01,NaN,30.3880,...,4.330,15.00,0.058,1.330,6.00,94.0,143.0,87.0,2022-09-01,1
3,1009462,1938.0,1,1001.0,-7.0,2004-01-01,NaN,2021-02-04,2021-02-04,32.2385,...,4.800,15.50,0.084,1.750,8.20,77.0,141.0,79.0,2021-02-04,1
4,1010028,1947.0,0,1001.0,-7.0,NaN,NaN,2019-04-11,2019-04-11,32.0399,...,4.440,13.40,0.042,0.950,7.10,71.0,131.0,55.0,2019-04-11,1


### Post process dataframe by converting data into common data types

In [8]:
### Save dataframe as csv, to manually observe the data
df = pd.read_csv("vascular_dementia_filtered.csv")

# convert field which use write date a disease is reported and mark it as true or false
# to determine if a person was diagnosed with the diseas or not rather than when.
DateReportedFields = [
    "Primary Hypertension",
    "Secondary Hypertension",
    "Report of stroke",
    "Stress Reported",
    "Seropositive Rheumatoid Arthritis",
    "Other Rheumatoid Arthritis",
    "Juvenile Arthritis",
    "Other Arthritis",
    "Psoriatic and enteropathic arthropathies",
    "Multiple Sclerosis",
    "Crohn's disease",
    "Ulcerative Colitis",
    "Thyrotoxicosis (Grave's disease)",
    "Sjogren Disease (M35)",
    "Myasthenia gravis",
    "Diagnosed with Coeliac disease",
    "B12 deficiency anaemia"
]

df = convert_date_to_binary(df, DateReportedFields)



#### Measure the distribution of age, education, and ethnicity of patients diagnosed with vascular dementia (or all columns)

In [9]:
df.describe()

,Unnamed: 0,id,Birth Year,Sex,Ethnicity,Education,Primary Hypertension,Secondary Hypertension,BMI Impedance,Smoking Status,...,Platelet distribution width,Red blood cell (erythrocyte) count,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Has Vascular Dementia
count,2186.000000,2.186000e+03,2186.000000,2186.000000,2181.000000,2137.000000,2186.000000,2186.000000,2162.000000,2181.000000,...,2037.000000,2037.000000,2037.000000,2005.000000,2005.000000,2037.000000,1983.000000,1983.000000,1983.000000,2186.0
mean,1092.500000,3.515518e+06,1943.111162,0.582342,1072.196240,-1.001404,0.789113,0.002745,28.557839,0.658414,...,16.542081,4.489278,13.725449,0.064314,1.435638,7.335621,81.993444,147.228946,70.938477,1.0
std,631.188165,1.442844e+06,4.317173,0.493286,468.249479,4.960943,0.408032,0.052330,5.139395,0.769999,...,0.531861,0.444468,1.094076,0.035324,0.804452,1.956365,11.199929,21.110899,13.162657,0.0
min,0.000000,1.002012e+06,1936.000000,0.000000,-3.000000,-7.000000,0.000000,0.000000,15.757600,-3.000000,...,15.200000,1.720000,11.490000,0.012000,0.263000,2.370000,47.000000,78.000000,35.000000,1.0
25%,546.250000,2.300567e+06,1940.000000,0.000000,1001.000000,-7.000000,1.000000,0.000000,25.062350,0.000000,...,16.200000,4.200000,13.100000,0.045000,1.020000,6.020000,74.000000,132.000000,62.000000,1.0
50%,1092.500000,3.511698e+06,1942.000000,1.000000,1001.000000,1.000000,1.000000,0.000000,27.794000,1.000000,...,16.500000,4.484000,13.530000,0.060000,1.330000,7.120000,82.000000,146.000000,70.000000,1.0
75%,1638.750000,4.741248e+06,1945.000000,1.000000,1001.000000,3.000000,1.000000,0.000000,31.421725,1.000000,...,16.860000,4.773000,14.100000,0.078000,1.740000,8.410000,89.000000,160.000000,79.000000,1.0
max,2185.000000,6.022126e+06,1968.000000,1.000000,4003.000000,6.000000,1.000000,1.000000,54.528300,2.000000,...,19.400000,6.390000,31.700000,1.077000,25.278000,34.130000,132.000000,241.000000,169.000000,1.0


#### Extract uniform distribution of patients who adher to the distribution of patients diagnosed with vascular dementia but are not diagnosed with it

### Save the extracted dataset

In [ ]:
df.to_csv('vascular_dementia_filtered_2.csv')

df.head(10)